<a href="https://colab.research.google.com/github/LaJeremi/Do-it-study-Lee-gi-chang-/blob/main/15_03_%EC%96%91%EB%B0%A9%ED%96%A5_LSTM%EA%B3%BC_%EC%96%B4%ED%85%90%EC%85%98_%EB%A9%94%EC%BB%A4%EB%8B%88%EC%A6%98(BiLSTM_with_Attention_mechanism).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. IMDB 리뷰 데이터 전처리하기




단뱡항 LSTM으로 텍스트 분류를 수행할 수도 있지만 때로는 양방향 LSTM을 사용하는 것이 더 강력합니다. 

여기에 추가적으로 어텐션 메커니즘을 사용할 수도 있습니다. 

양방향 LSTM과 어텐션 메커니즘으로 IMDB 리뷰 감성 분류하기를 수행해봅시다.

In [ ]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

IMDB 리뷰 데이터는 앞서 텍스트 분류하기 챕터에서 다룬 바 있으므로 데이터에 대한 상세 설명은 생략합니다. 최대 단어 개수를 10,000으로 제한하고 훈련 데이터와 테스트 데이터를 받아옵니다.

In [ ]:
vocab_size = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

17464789/17464789 [==============================] - 0s 0us/step


훈련 데이터와 이에 대한 레이블이 각각 X_train, y_train에 테스트 데이터와 이에 대한 레이블이 각각 X_test, y_test에 저장되었습니다. IMDB 리뷰 데이터는 이미 정수 인코딩이 된 상태므로 남은 전처리는 패딩뿐입니다. 리뷰의 최대 길이와 평균 길이를 확인해봅시다.

In [ ]:
print('리뷰의 최대 길이 : {}'.format(max(len(l) for l in X_train)))
print('리뷰의 평균 길이 : {}'.format(sum(map(len, X_train))/len(X_train)))

리뷰의 최대 길이 : 2494
리뷰의 평균 길이 : 238.71364


리뷰의 최대 길이는 2,494이며 리뷰의 평균 길이는 약 238로 확인됩니다. 평균 길이보다는 조금 크게 데이터를 패딩하겠습니다.



In [ ]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

훈련용 리뷰와 테스트용 리뷰의 길이가 둘 다 500이 되었습니다.



# 2. 바다나우 어텐션(Bahdanau Attention)


In [ ]:
import tensorflow as tf


In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = Dense(units)
    self.W2 = Dense(units)
    self.V = Dense(1)

  def call(self, values, query): # 단, key와 value는 같음
    # query shape == (batch_size, hidden size)
    # hidden_with_time_axis shape == (batch_size, 1, hidden size)
    # score 계산을 위해 뒤에서 할 덧셈을 위해서 차원을 변경해줍니다.
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # attention_weights shape == (batch_size, max_length, 1)
    attention_weights = tf.nn.softmax(score, axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

# 3. 양방향 LSTM + 어텐션 메커니즘(BiLSTM with Attention Mechanism)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

이제 모델을 설계해보겠습니다. 여기서는 케라스의 함수형 API를 사용합니다. 우선 입력층과 임베딩층을 설계합니다.



In [ ]:
sequence_input = Input(shape=(max_len,), dtype='int32')
embedded_sequences = Embedding(vocab_size, 128, input_length=max_len, mask_zero = True)(sequence_input)

10,000개의 단어들을 128차원의 벡터로 임베딩하도록 설계하였습니다. 이제 양방향 LSTM을 설계합니다. 단, 여기서는 양방향 LSTM을 두 층을 사용하겠습니다. 우선, 첫번째 층입니다. 두번째 층을 위에 쌓을 예정이므로 return_sequences를 True로 해주어야 합니다.

In [ ]:
lstm = Bidirectional(LSTM(64, dropout=0.5, return_sequences = True))(embedded_sequences)


두번째 층을 설계합니다. 상태를 리턴받아야 하므로 return_state를 True로 해주어야 합니다.



In [ ]:
lstm, forward_h, forward_c, backward_h, backward_c = Bidirectional \
  (LSTM(64, dropout=0.5, return_sequences=True, return_state=True))(lstm)

각 상태의 크기(shape)를 출력해보겠습니다.



In [ ]:
print(lstm.shape, forward_h.shape, forward_c.shape, backward_h.shape, backward_c.shape)


(None, 500, 128) (None, 64) (None, 64) (None, 64) (None, 64)


순방향 LSTM의 은닉 상태와 셀상태를 forward_h, forward_c에 저장하고, 

역방향 LSTM의 은닉 상태와 셀 상태를 backward_h, backward_c에 저장합니다.

각 은닉 상태나 셀 상태의 경우에는 128차원을 가지는데, 
lstm의 경우에는 (500 × 128)의 크기를 가집니다.

 foward 방향과 backward 방향이 연결된 hidden state벡터가 
 모든 시점에 대해서 존재함을 의미합니다.

양방향 LSTM을 사용할 경우에는 순방향 LSTM과 역방향 LSTM 각각 은닉 상태와 

셀 상태를 가지므로, 양방향 LSTM의 은닉 상태와 셀 상태를 사용하려면 

두 방향의 LSTM의 상태들을 연결(concatenate)해주면 됩니다.

In [ ]:
state_h = Concatenate()([forward_h, backward_h]) # 은닉 상태
state_c = Concatenate()([forward_c, backward_c]) # 셀 상태

어텐션 메커니즘에서는 은닉 상태를 사용합니다. 이를 입력으로 컨텍스트 벡터(context vector)를 얻습니다.



In [ ]:
attention = BahdanauAttention(64) # 가중치 크기 정의
context_vector, attention_weights = attention(lstm, state_h)

컨텍스트 벡터를 밀집층(dense layer)에 통과시키고, 이진 분류이므로 최종 출력층에 1개의 뉴런을 배치하고, 활성화 함수로 시그모이드 함수를 사용합니다.

In [ ]:
dense1 = Dense(20, activation="relu")(context_vector)
dropout = Dropout(0.5)(dense1)
output = Dense(1, activation="sigmoid")(dropout)
model = Model(inputs=sequence_input, outputs=output)

옵티마이저로 아담 옵티마이저 사용하고, 모델을 컴파일합니다.



In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


시그모이드 함수를 사용하므로 손실 함수로 binary_crossentropy를 사용하였습니다. 이제 모델을 훈련하겠습니다.



In [ ]:
history = model.fit(X_train, y_train, epochs = 3, batch_size = 256, validation_data=(X_test, y_test), verbose=1)


Epoch 1/3
98/98 [==============================] - 1013s 10s/step - loss: 0.4781 - accuracy: 0.7687 - val_loss: 0.2891 - val_accuracy: 0.8806
Epoch 2/3
98/98 [==============================] - 964s 10s/step - loss: 0.2420 - accuracy: 0.9138 - val_loss: 0.2878 - val_accuracy: 0.8820
Epoch 3/3
98/98 [==============================] - 978s 10s/step - loss: 0.1872 - accuracy: 0.9365 - val_loss: 0.4092 - val_accuracy: 0.8529


검증 데이터로 테스트 데이터를 사용하여 에포크가 끝날 때마다 테스트 데이터에 대한 정확도를 출력하도록 하였습니다.



In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test)[1]))


782/782 [==============================] - 283s 361ms/step - loss: 0.4092 - accuracy: 0.8529

 테스트 정확도: 0.8529
